In [24]:
import sqlite3
from tqdm import tqdm
import vk
import time
import pandas as pd

In [25]:
#open vk session
with open('token') as f:
    TOKEN = f.read()

with open('uid') as f: 
    SELF_ID = f.read()
    
SLEEP_TIME = 0.3

session = vk.Session(access_token=TOKEN)
vkapi = vk.API(session)

In [26]:
with open('data/uids.txt') as f: 
    users = f.read().split('\n')

In [27]:
len(users)

153

In [42]:
df = pd.read_csv('data/survey_data.csv', sep=';', decimal=',')
users = list(df.id)

In [54]:
#database
conn = sqlite3.connect('users.db')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS uids (uid STRING)');
c.execute('CREATE TABLE IF NOT EXISTS posts (owner_id STRING, date STRING, text STRING)');

In [55]:
#uids table
c.executemany('INSERT INTO uids VALUES (?)', [(str(u),) for u in users])
conn.commit()

In [46]:
%%time
def get_post_count(friend):
    posts = vkapi('wall.get', owner_id=friend, count=1, filter='owner', v=5.73)
    return posts['count']

drop = 0
for user in tqdm(users, desc='Downloading posts'):
    try:
        current_post_count = get_post_count(user)
        time.sleep(SLEEP_TIME)
        if (current_post_count > 7000) or (current_post_count < 100):
            continue
        current_posts = []
        resid = current_post_count
        offset = 0
        while resid > 0:
            response = vkapi('wall.get', owner_id=user, count=100, filter='owner', 
                             offset=offset, v=5.73)['items']
            time.sleep(SLEEP_TIME)
            current_posts += response
            resid -= 100
            offset += 100
            chunk = []
            for p in current_posts:
                if type(p) == dict:
                    chunk.append((p['owner_id'], p['date'], p['text']))
            c.executemany('INSERT INTO posts VALUES (?,?,?)', chunk)
            conn.commit()
    except:
        drop += 1
        continue
print(drop, 'пользователей потеряно')

Wall time: 18min 56s


In [56]:
c.close()
conn.close()